# SnakeQL
# An SQL like Database built entirely within Python

In [2]:
#libraries imported
import json
import os
import threading
import datetime
from decimal import Decimal
import ply.lex as lex
import ply.yacc as yacc

In [3]:
class DatabaseError(Exception):
    pass

In [4]:
class TableExistsError(DatabaseError):
    pass

In [5]:
class ColumnNotFoundError(DatabaseError):
    pass

In [6]:
class DataTypeError(DatabaseError):
    pass

In [7]:
class TransactionError(DatabaseError):
    pass

In [8]:
class Transaction:
    def __init__(self, isolation_level='READ COMMITTED'):
        self.operations = []
        self.active = False
        self.isolation_level = isolation_level